# Number of Possible Neighbors

In [1]:
import os
from pathlib import Path
import pandas as pd
from ast import literal_eval

import warnings

CURR_PATH = Path.cwd()
SEED = 59

warnings.filterwarnings('ignore')
font_family = "Calibri"

data_path = CURR_PATH.parents[0] / 'data'

PROGS = ['AR', 'CM', 'PT']

In [2]:
styles = [
    dict(selector="th", props=[("font-size", "11pt"),
                               ("text-align", "center"),
                               ('font-family', font_family),
                               ('background-color', 'white'),
                               ('border-style', 'solid'),
                               ('border-color', 'grey'),
                               ('border-width', 'thin')]),
    dict(selector="td", props=[("font-size", "11pt"),
                               #("text-align", "center"),
                               ('font-family', font_family),
                               #('background-color', 'white'),
                               ('border-style', 'solid'),
                               ('border-color', 'grey'),
                               ('border-width', 'thin')]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]

## Recommendations Data

In [3]:
recoms = pd.read_csv(data_path / 'recom_data.csv', sep=';')

set_cols = [
    'Passed_Past', 'Passed_Next', 
    'ID_TO', 'D_List', 'Count_Next_TO',
    'Nbrs_Passed_Next_TO', 'AC', 'No_Recom_Rule', 'R', 'C', 'M', 'W', 'NoR',
    'NR'
]

for c in set_cols:
    recoms[c] = recoms[c].replace('set()',
                                  '{}').replace('None',
                                                '{}').apply(literal_eval)

print(recoms.shape, recoms['T'].unique())
recoms.sample(n=5, random_state=SEED)

(16392, 64) ['AN' 'AN1' 'AN2' 'AN3' 'AN4' 'GN']


,P,ID,ST,#E_1,#E_2,#E_3,#F_1,#F_2,#F_3,#NE_1,...,ACC,REC_1,REC_0,BACC,PREC_1,F1,MCC,NT,MPT,PS
8464,CM,1974,G,5.0,6.0,6.0,0.0,0.0,0.0,0.0,...,1.000000,1.0,0.0,0.5,1.000000,1.0,0.0,AN,P4,CM2
7264,CM,421,D,5.0,5.0,7.0,1.0,0.0,2.0,0.0,...,1.000000,1.0,0.0,0.5,1.000000,1.0,0.0,AN,P4,CM2
9111,CM,2788,D,5.0,6.0,6.0,0.0,1.0,2.0,0.0,...,0.666667,0.8,0.0,0.4,0.800000,0.8,-0.2,AN,P3,CM2
1898,AR,1937,G,5.0,6.0,6.0,0.0,0.0,0.0,0.0,...,0.666667,1.0,0.0,0.5,0.666667,0.8,0.0,AN,P2,AR2
1373,AR,1477,G,5.0,6.0,6.0,0.0,0.0,1.0,0.0,...,1.000000,1.0,0.0,0.5,1.000000,1.0,0.0,GN,P0,AR2


## All Students

In [16]:
# get the overall numbers of students for each program
ct_all = pd.crosstab(recoms['P'],
                 recoms['ST'],
                 values=recoms['ID'],
                 aggfunc='nunique',
                 margins=True)
ct_all

ST,D,G,All
P,,,
AR,134,444,578
CM,221,306,527
PT,58,203,261
All,413,953,1366


## 2nd Semester

In [17]:
# get the number of students by number of passed courses

data = recoms[recoms['S'] == 2]

ct_2 = pd.crosstab(data['P'],
                   data['#P_2'],
                   values=data['ID'],
                   aggfunc='nunique',
                   margins=True).fillna(0)
ct_2 = ct_2.drop(index='All')

# get the difference of all students and 
# the number of students according the minimum number of passed courses

for p in PROGS:
    cums = []
    cum = ct_2.loc[p, 'All']
    cums.append(cum)
    
    for x in [c for c in ct_2.columns]:
        if x != 'All':
            val = ct_2.loc[p,x]
            cum -=val
            cums.append(cum)
            
    ct_2.loc[f'{p}', :] = cums
    
    
ct_2 = ct_2.sort_index()#.columns
ct_2 = ct_2.drop(columns='All')
#ct_2.loc['All',:] = ct_2.sum()

ct_2

#P_2,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
P,,,,,,,,,,
AR,578.0,560.0,538.0,506.0,457.0,358.0,232.0,3.0,0.0,0.0
CM,527.0,454.0,414.0,382.0,328.0,258.0,142.0,10.0,2.0,1.0
PT,261.0,247.0,235.0,227.0,213.0,165.0,52.0,3.0,1.0,1.0


In [18]:
# merge the crosstabs and reorder the columns

ct_all_2 = pd.concat([ct_all, ct_2], axis=1).fillna(0).astype(int)#.sort_index(axis=1)
ct_all_2 = ct_all_2.reset_index().set_index(['P','All', 'D', 'G'])

ct_all_2['S'] = 2
ct_all_2 = ct_all_2[['S'] + [c for c in ct_all_2.columns if c != 'S']]
ct_all_2 = ct_all_2.drop(index='All')
ct_all_2

,,,,S,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
P,All,D,G,,,,,,,,,,,
AR,578,134,444,2,578,560,538,506,457,358,232,3,0,0
CM,527,221,306,2,527,454,414,382,328,258,142,10,2,1
PT,261,58,203,2,261,247,235,227,213,165,52,3,1,1


## 3rd Semester

In [19]:
# get the number of students by number of passed courses

data = recoms[recoms['S'] == 3]

ct_3 = pd.crosstab(data['P'],
                   data['#P_3'],
                   values=data['ID'],
                   aggfunc='nunique',
                   margins=True).fillna(0)
ct_3 = ct_3.drop(index='All')


# get the difference of all students and 
# the number of students according the minimum number of passed courses

for p in PROGS:
    cums = []
    cum = ct_3.loc[p, 'All']
    cums.append(cum)
    
    for x in [c for c in ct_3.columns]:
        if x != 'All':
            val = ct_3.loc[p,x]
            cum -=val
            cums.append(cum)
            
    ct_3.loc[f'{p}', :] = cums
    
    
ct_3 = ct_3.sort_index()
ct_3 = ct_3.drop(columns='All')
ct_3

#P_3,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0
P,,,,,,,,,,
AR,578.0,530.0,506.0,477.0,441.0,382.0,273.0,33.0,1.0,0.0
CM,527.0,403.0,375.0,346.0,306.0,235.0,119.0,16.0,4.0,1.0
PT,261.0,236.0,230.0,220.0,206.0,181.0,145.0,29.0,5.0,0.0


In [21]:
# merge the crosstabs and reorder the columns

ct_all_3 = pd.concat([ct_all, ct_3], axis=1).fillna(0).astype(int)
ct_all_3 = ct_all_3.reset_index().set_index(['P','All', 'D', 'G'])

ct_all_3['S'] = 3
ct_all_3 = ct_all_3[['S'] + [c for c in ct_all_3.columns if c != 'S']]
ct_all_3 = ct_all_3.drop(index='All')

ct_all_3

,,,,S,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0
P,All,D,G,,,,,,,,,,,
AR,578,134,444,3,578,530,506,477,441,382,273,33,1,0
CM,527,221,306,3,527,403,375,346,306,235,119,16,4,1
PT,261,58,203,3,261,236,230,220,206,181,145,29,5,0


## Merge 2nd and 3rd Semester

In [22]:
nbr_neigh = pd.concat([ct_all_2, ct_all_3]).fillna(0).astype(int).reset_index()

nbr_neigh = nbr_neigh.set_index(['P', 'All', 'D', 'G', 'S']).sort_index().drop(columns=[6,7,8,9,10])

nbr_neigh_styled = nbr_neigh.style.set_table_styles(styles)\
    .format_index(axis=1, precision=0) \
    .format('{:.0f}')
    
nbr_neigh_styled